In [1]:
from elasticsearch import Elasticsearch
from elasticsearch.helpers import streaming_bulk
import glob
import ujson as json
from tqdm import tqdm
import os
import pandas as pd
from typing import Any, Dict, Iterable


In [51]:
# with open('stopwords.txt', 'r') as f:
#     stopwords_list = f.read().split("\n")


def create_fund_indx(es: Elasticsearch, doc_dims: int = 256):
    dense_funds = {
        "settings": {
            "analysis": {
                "analyzer": {
                    "pl_analyzer": {
                        "type": "standard",
                        "filter": ["lowercase", "polish_stop", "polish_stem"],
                        "char_filter": ["html_strip"]
                    },
                },
                "filter": {
                    "polish_stop": {
                        "type": "stop",
                                "ignore_case": True,
                                "stopwords": ["_polish_"]  # plugin required
                    }
                }
            },
        },
        "mappings": {
            "properties": {
                # "text_vector": {
                #     "type": "dense_vector",
                #     "dims": doc_dims
                # },
                "description": {
                    "type": "text",
                    "analyzer": "polish"
                },
                "name": {
                    "type": "keyword"
                },
                "url": {
                    "type": "text"
                }
            }
        }
    }
    es.indices.delete("test", ignore=[400, 404])
    es.indices.create(index="test", body=dense_funds)


def strip_extra_tags(txt):
    for tag in ('\n',
                r'<em>', r'</em>', "\t"
                ):
        txt = txt.replace(tag, "")
    txt = txt.replace(u'\xa0', u' ')
    return txt


def stream_fund_docs(src_df: pd.DataFrame) -> Iterable[Dict[str, Any]]:
    """Stream funds to the ES stack"""
    records = src_df.to_dict('records')
    for record in records:
        yield {
            "_index": "test",
            "_source": {
                'description': strip_extra_tags(record['FullDesc']),
                'name': record['Name'],
                'URL': record['URL']
                # "source": "test"
            }
        }


In [52]:
from dotenv import load_dotenv, find_dotenv
load_dotenv(find_dotenv())

user = os.environ.get("USER")
password = os.environ.get("PASS")
addr = os.environ.get("ADDR")

src_df = pd.read_pickle('../backend/data/PFR.pkl')
es = Elasticsearch(
    hosts=[
        f"http://{user}:{password}@{addr}:9200"
    ]

)
create_fund_indx(es)
stream = stream_fund_docs(src_df=src_df)
for ok, response in streaming_bulk(es, actions=stream):
    if not ok:
        print(response)


/opt/homebrew/lib/python3.9/site-packages/elasticsearch/connection/base.py:208: ElasticsearchWarning: Elasticsearch built-in security features are not enabled. Without authentication, your cluster could be accessible to anyone. See https://www.elastic.co/guide/en/elasticsearch/reference/7.13/security-minimal-setup.html to enable security.
  warnings.warn(message, category=ElasticsearchWarning)


In [53]:
def simple_query(PKD: str):
    query = {
        'query': {
            'simple_query_string': {
                'query': PKD,
                "fields": ["description"]
            }
        },
        "highlight": {
            "fields": {
                "description": {}  # highlight here
            }
        }
    }

    results = es.search(index='test', body=query)
    return results['hits']['hits']


def es_search(PKD: str):
    query = {
        'query': {
            "bool": {
                "must": {
                    "query_string": {
                        "query": PKD
                    }
                },
                # "should": [
                #     {"term": {"tags": "lat"}}
                # ],
                # "minimum_should_match": 1
            },
        },
        "highlight": {
            "fields": {
                "description": {}
            }
        }
    }
    results = es.search(index='test', body=query)
    return results['hits']['hits']

simple_query("działalność związana z wytwarzaniem oprogramowania")

/opt/homebrew/lib/python3.9/site-packages/elasticsearch/connection/base.py:208: ElasticsearchWarning: Elasticsearch built-in security features are not enabled. Without authentication, your cluster could be accessible to anyone. See https://www.elastic.co/guide/en/elasticsearch/reference/7.13/security-minimal-setup.html to enable security.
  warnings.warn(message, category=ElasticsearchWarning)


[{'_index': 'test',
  '_type': '_doc',
  '_id': 'aZI0qn0BT59karurJ0Hq',
  '_score': 4.913625,
  '_source': {'description': ' Granty na transfer technologii – Sieć Otwartych Innowacji Do góry +48 22 703 43 00 A A en Dla biznesu Dla startupów Dla mikroprzedsiębiorców Dla firmy zagranicznej Dla MŚP Dla dużej firmy Dla samorządów Dla Ciebie Dla gospodarki Tarcza Antykryzysowa Ekspertyzy ekonomiczne PFR dla Miast Inwestycje w nieruchomości mieszkaniowe PFR Green Hub Pracownicze Plany Kapitałowe Start in Poland O nas Instytucje rozwoju PFR S.A. PFR Ventures PFR TFI PFR Nieruchomości PFR Portal PPK ARP BGK KUKE KUKE Finance PAIH PARP Działalność społeczna Fundacja PFR Centralny Dom Technologii Fundacja BGK Fundacja ARP O Grupie PFR Kim jesteśmy Rada Grupy PFR Aktualności Blog Kariera Dla mediów Kontakt × × Dla biznesu Dla biznesu Dla startupów Dla mikroprzedsiębiorców Dla firmy zagranicznej Dla MŚP Dla dużej firmy Dla samorządów Dla Ciebie Dla gospodarki Dla gospodarki Tarcza Antykryzysowa Ek

In [18]:
import pandas as pd


# df = pd.read_csv('../BKG/bgk.tsv', sep='\t')
# df.to_pickle('../backend/data/BGK.pkl')
df = pd.read_csv('../PFR/pfr.tsv', sep='\t')
df.rename(columns={
    "name": "Name",
    "text": "FullDesc",
    "url": "URL"
}, inplace=True
)
df.to_pickle('../backend/data/PFR.pkl')
# df.to_pickle('../backend/data/BGK.pkl')


In [27]:
from dotenv import load_dotenv, find_dotenv
load_dotenv(find_dotenv())

user = os.environ.get("USER")
password = os.environ.get("PASS")
addr = os.environ.get("ADDR")

# src_df = pd.read_pickle('../pfr/PFR_free_txt.pkl')
es = Elasticsearch(
    hosts=[
        f"http://{user}:{password}@{addr}:9200"
    ]
)


def simple_query(PKD: str):
    query = {
        'query': {
            'simple_query_string': {
                'query': PKD,
                "fields": ["description"]
            }
        },
        "highlight": {
            "fields": {
                "description": {}  # highlight here
            }
        }
    }

    results = es.search(index='funds', body=query)
    return results['hits']['hits']


simple_query(
    "inclusion"
)


/opt/homebrew/lib/python3.9/site-packages/elasticsearch/connection/base.py:208: ElasticsearchWarning: Elasticsearch built-in security features are not enabled. Without authentication, your cluster could be accessible to anyone. See https://www.elastic.co/guide/en/elasticsearch/reference/7.13/security-minimal-setup.html to enable security.
  warnings.warn(message, category=ElasticsearchWarning)


[]

In [ ]:
!http://localhost:9200/foo/_search?pretty=true&q=*:*